In [1]:
import numpy as np
import math
from scipy import integrate
import matplotlib.pyplot as plt
import random
import time
from scipy.stats import rankdata

In [2]:
def lap_noise(data, epsilon, K, N, M):
    pri_data = data + np.random.laplace(loc = 0.0, scale = (K*16*(N-1)/N)/epsilon, size=data.shape)
    
    min = 10000
    
    for i in range(M):
        if pri_data[i] > 0 and pri_data[i] < min:
            min = pri_data[i]
    
    for i in range(M):
        if pri_data[i] < 0:
            pri_data[i] = min
    
    return pri_data

In [3]:
def td(n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    
    if h == 0:
        return 0
    else:
        return (2*((i-j)**2))/h

def SHD_td(cc,n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    T = td(n)
    
    d = np.zeros(2)
    N = np.zeros(10)
    
    for k in range(10):
        N[k] = n[k]
    
    if T < cc:
        while T < cc:
            if N[6] > 0: N[6] -= 1
            elif N[5] > 0: N[5] -= 1
            elif N[2] > 0: N[2] -= 1
            elif N[4] > 0: N[4] -= 1
            elif N[8] > 0: N[8] -= 1
            elif N[1] > 0: N[1] -= 1
            elif N[0] > 0: N[0] -= 1
            elif N[7] > 0: N[7] -= 1
            else: N[3] -= 1
            N[9] += 1
            
            T = td(N)
            d[0] -= 1
        
        T = td(n)
        for k in range(10):
            N[k] = n[k]
        
        while T < cc:
            if N[9] > 0: N[9] -= 1
            elif N[7] > 0: N[7] -= 1
            elif N[3] > 0: N[3] -= 1
            elif N[4] > 0: N[4] -= 1
            elif N[8] > 0: N[8] -= 1
            elif N[1] > 0: N[1] -= 1
            elif N[0] > 0: N[0] -= 1
            elif N[5] > 0: N[5] -= 1
            else: N[2] -= 1
            N[6] += 1
            
            T = td(N)
            d[1] -= 1
        
        if d[0] > d[1]:
            return d[0]
        else:
            return d[1]
        
    else:
        step = 0
        
        for k in range(10):
            N[k] = n[k]
            
        if i > j:
            while T >= cc:
                if N[9] > 0: N[9] -= 1
                elif N[3] > 0: N[3] -= 1
                elif N[7] > 0: N[7] -= 1
                elif N[0] > 0: N[0] -= 1
                elif N[1] > 0: N[1] -= 1
                elif N[4] > 0: N[4] -= 1
                elif N[8] > 0: N[8] -= 1
                elif N[2] > 0: N[2] -= 1
                else: N[5] -= 1
                N[6] += 1
                
                T = td(N)
                step += 1
        else:
            while T >= cc:
                if N[6] > 0: N[6] -= 1
                elif N[2] > 0: N[2] -= 1
                elif N[5] > 0: N[5] -= 1
                elif N[0] > 0: N[0] -= 1
                elif N[1] > 0: N[1] -= 1
                elif N[4] > 0: N[4] -= 1
                elif N[8] > 0: N[8] -= 1
                elif N[3] > 0: N[3] -= 1
                else: N[7] -= 1
                N[9] += 1
                
                T = td(N)
                step += 1
                
        return step-1

In [4]:
def appx_SHD_td(cc,n):
    h = n[1]+n[2]+n[3] + 2*(n[4]+n[5]+n[6]+n[7]+n[8]+n[9])
    i = n[3]+n[7]+n[8] + 2*n[9]
    j = n[2]+n[5] + 2*n[6] + n[8]
    T = td(n)
    
    if T < cc:
        if h <= cc/2:
            return -math.ceil((cc-h-math.fabs(i-j))/4)
        else:
            return -math.ceil((math.sqrt(h*cc/2)-math.fabs(i-j))/4)
    else:
        return math.ceil((math.fabs(i-j)-math.sqrt(h*cc/2))/4) - 1

In [5]:
def random_num(pd):
    dist = np.cumsum(pd).tolist()
    dist[-1] = 1.0
    num = np.random.rand()
    dist.append(num)
    return sorted(dist).index(num)

def exp_mec(shd, m, K, epsilon):
    S = np.zeros(K)
    k = 0
    
    w = np.zeros(m)
    p = np.zeros(m)
    
    while k < K:
        sumw = 0
        for i in range(m):
            w[i] = math.exp(epsilon * shd[i] / (2*K))
            sumw += w[i]
        for i in range(m):
            p[i] = w[i]/sumw
        
        x = random_num(p)
        shd[x] = -1000000.0
        S[k] = x
        k += 1
    
    return S

In [6]:
for j in range(5):
    N = 150
    M = 5000
    H = np.zeros(M)
    I = np.zeros(M)
    J = np.zeros(M)
    
    n = np.zeros((M,10))
    
    for i in range(M-10):
        for j in range(10):
            S = 0
            for k in range(j):
                S += n[i][k]
            n[i][j] = np.random.binomial(2*N-S, 1/(10-j))
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
    
    for i in range(M-10,M):
        for j in range(10):
            S = 0
            for k in range(j):
                S += n[i][k]
            if j == 2 or j == 5 or j == 6:
                n[i][j] = np.random.binomial(2*N-S, 1.6/(10-j))
            else:
                n[i][j] = np.random.binomial(2*N-S, 1/(10-j))
        H[i] = n[i][1]+n[i][2]+n[i][3] + 2*(n[i][4]+n[i][5]+n[i][6]+n[i][7]+n[i][8]+n[i][9])
        I[i] = n[i][3]+n[i][7]+n[i][8] + 2*n[i][9]
        J[i] = n[i][2]+n[i][5] + 2*n[i][6] + n[i][8]
    
    stats = np.zeros(M)

    for i in range(M):
        if H[i] == 0:
            stats[i] = 0
        else:
            stats[i] = (2*((I[i]-J[i])**2))/H[i]
    
    shd = np.zeros(M)
    appx_shd = np.zeros(M)
    cc = 19.5
    epsilon = 3
    K = 3
    
    lap_start = time.time()
    lap_noise(stats, epsilon, K, N, M)
    lap_end = time.time()
    
    print("lap_time =",lap_end-lap_start,"[sec]")
    
    appx_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5]), int(n[i][6]), int(n[i][7]), int(n[i][8]), int(n[i][9])]
        appx_shd[i] = appx_SHD_td(cc,k)
    exp_mec(appx_shd, M, K, epsilon)
    appx_end = time.time()
    
    print("appx_time =",appx_end-appx_start,"[sec]")
    
    exact_start = time.time()
    for i in range(M):
        k = [int(n[i][0]), int(n[i][1]), int(n[i][2]), int(n[i][3]), int(n[i][4]), int(n[i][5]), int(n[i][6]), int(n[i][7]), int(n[i][8]), int(n[i][9])]
        shd[i] = SHD_td(cc,k)
    exp_mec(shd, M, K, epsilon)
    exact_end = time.time()
    
    print("exact_time =",exact_end-exact_start,"[sec]")
    print("----------")

lap_time = 0.00379180908203125 [sec]
appx_time = 0.05003714561462402 [sec]
exact_time = 1.1837742328643799 [sec]
----------
lap_time = 0.0037031173706054688 [sec]
appx_time = 0.04920601844787598 [sec]
exact_time = 1.1705670356750488 [sec]
----------
lap_time = 0.003856182098388672 [sec]
appx_time = 0.04927682876586914 [sec]
exact_time = 1.1946988105773926 [sec]
----------
lap_time = 0.0037467479705810547 [sec]
appx_time = 0.05018019676208496 [sec]
exact_time = 1.1754019260406494 [sec]
----------
lap_time = 0.0036971569061279297 [sec]
appx_time = 0.04944300651550293 [sec]
exact_time = 1.165079116821289 [sec]
----------
